# Get API Key and libraries

In [26]:
import openai
import os
import json
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from IPython.display import display, Markdown, Latex, HTML, JSON

# Open the JSON file
with open('/Users/edwin/Repositories/api_keys.json') as file:
    data = json.load(file)

# Access the API keys
openai.api_key= data['apiKeys']['OpenAI']

# Define Functions

### get_completion to send direct messages to gpt endpoint

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

### Function for analyzing emails

First we define the instructions for receiving a json file as output.

In [11]:
Sender = ResponseSchema(name="Sender",
                       description="The name of the sender. If this information is not found, output unknown.")

Tone = ResponseSchema(name="Tone",
                    description="Give me the tone. It can be negative, neutral or positive.")

Deliverables = ResponseSchema(name="Deliverables",
                                    description="Are there any deliverables in form of files that the sender is expecting from me to send?. If not, output No. If so, give me the names of the files expected to send with the file extension expected in \
              parenthesis. Everything sould be comma separated.")

Number_of_deliverables = ResponseSchema(name="Number of deliverables",
                    description="Give me a list of the most important 5 tags to classify the email separated by commas")

Tags = ResponseSchema(name="Tags",
                    description="Give me the tone. It can be negative, neutral or positive.")

Importance = ResponseSchema(name="Importance",
                    description="Tell me the level of importance. Can be not important, medium, important and very important.")

Summary = ResponseSchema(name="Summary",
                    description="Give me a brief description of what's on the email.")

Translation = ResponseSchema(name="Translation",
                    description="Translate the summary of the email to spanish.")


response_schemas = [Sender, 
                    Tone,
                    Deliverables,
                    Number_of_deliverables,
                    Tags,
                    Importance,
                    Summary,
                    Translation
                    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Sender": string  // The name of the sender. If this information is not found, output unknown.
	"Tone": string  // Give me the tone. It can be negative, neutral or positive.
	"Deliverables": string  // Are there any deliverables in form of files that the sender is expecting from me to send?. If not, output No. If so, give me the names of the files expected to send with the file extension expected in               parenthesis. Everything sould be comma separated.
	"Number of deliverables": string  // Give me a list of the most important 5 tags to classify the email separated by commas
	"Tags": string  // Give me the tone. It can be negative, neutral or positive.
	"Importance": string  // Tell me the level of importance. Can be not important, medium, important and very important.
	"Summary": string  // Give me a brief description of what's on

Then we need to give the email to analyze

In [12]:
email="""
Hi Edwin,

 

Congratulations! We are so glad you will be joining our team as a Business Analyst. Your start date is scheduled to be Monday July 3rd.

 

You will be receiving a separate email from Deel inviting you to review & sign your contract. Deel is the online platform we utilize for payments and to sending contracts compliantly. Attached is a tutorial on how to view the contract. Please look for the separate email from "Alicia from Deel" and reference the attached PDF for a step-by-step walkthrough. The video mentioned on page 1 of the document is very helpful.

 

Please also send back the following:

Staff Introduction: We will send out a new hire announcement on your first day. In preparation for this, could you please send back the following before EOD June 23rd:
A short bio written in 3rd person (See attached example. Please keep it to 1 or 2 paragraphs and be sure to highlight your subject matter expertise!)
1 headshot/candid photo (This will be used for your introduction and internal documents)
1 video of you waving! (This can be about 10 seconds long and shot on your phone vertically and emailed over to us. See the attached example of what we will put together)
Equipment: We will be connecting with our Tech Team to see what options are available for an extra SSD and will get back to you!
 

Please let us know if you have any questions. I’ll be sending more information about your first day the week before your start date. 😊 Thanks and congratulations!

 

 

stephanie reynolds | human resource & talent acquisition specialist | d. 201-216-4678
2 University Plaza Drive, Suite 406, Hackensack, NJ 07601

"""

Finally we define the prompt

In [13]:
template_email="""
For the following email, extract the following information:

Sender: The name of the sender. If this information is not found, output unknown.

Tone: Give me the tone. It can be negative, neutral or positive.

Deliverables: Are there any deliverables in form of files that the sender is expecting from me to send?. If not, output No. If so, give me the names of the files expected to send with the file extension expected in \
              parenthesis. Everything sould be comma separated.

Number of deliverables: Return the number of deliverables expected. If there are not, return 0.

Tags: Give me a list of the most important 5 tags to classify the email separated by commas. 

Importance: Tell me the level of importance. Can be not important, medium, important and very important.

Summary: Give me a brief description of what's on the email. 

Translation: Translate the summary of the email to spanish.

email: {email}

{format_instructions}

"""
prompt = ChatPromptTemplate.from_template(template=template_email)

messages = prompt.format_messages(email=email, 
                                format_instructions=format_instructions)

print(messages[0].content)


For the following email, extract the following information:

Sender: The name of the sender. If this information is not found, output unknown.

Tone: Give me the tone. It can be negative, neutral or positive.

Deliverables: Are there any deliverables in form of files that the sender is expecting from me to send?. If not, output No. If so, give me the names of the files expected to send with the file extension expected in               parenthesis. Everything sould be comma separated.

Number of deliverables: Return the number of deliverables expected. If there are not, return 0.

Tags: Give me a list of the most important 5 tags to classify the email separated by commas. 

Importance: Tell me the level of importance. Can be not important, medium, important and very important.

Summary: Give me a brief description of what's on the email. 

Translation: Translate the summary of the email to spanish.

email: 
Hi Edwin,

 

Congratulations! We are so glad you will be joining our team as a

In [14]:
chat = ChatOpenAI(temperature=0.0,openai_api_key=data['apiKeys']['OpenAI'])
response = chat(messages)
print(response.content)

```json
{
	"Sender": "Stephanie Reynolds",
	"Tone": "Positive",
	"Deliverables": "Staff Introduction Example (PDF), Headshot/Candid Photo (.jpg), Video of waving (.mp4)",
	"Number of deliverables": 3,
	"Tags": "New Hire, Onboarding, Contract, Deliverables, Start Date",
	"Importance": "Medium",
	"Summary": "The email is congratulating Edwin on joining the team as a Business Analyst and providing information about his start date. The email also includes instructions for reviewing and signing the contract, as well as requests for a staff introduction, headshot, and video of waving.",
	"Translation": "El correo electrónico felicita a Edwin por unirse al equipo como Analista de Negocios y proporciona información sobre su fecha de inicio. El correo electrónico también incluye instrucciones para revisar y firmar el contrato, así como solicitudes de una introducción del personal, una foto de cabeza y un video saludando."
}
```


In [22]:
output_dict = output_parser.parse(response.content)
output_dict

{'Sender': 'Stephanie Reynolds',
 'Tone': 'Positive',
 'Deliverables': 'Staff Introduction Example (PDF), Headshot/Candid Photo (.jpg), Video of waving (.mp4)',
 'Number of deliverables': 3,
 'Tags': 'New Hire, Onboarding, Contract, Deliverables, Start Date',
 'Importance': 'Medium',
 'Summary': 'The email is congratulating Edwin on joining the team as a Business Analyst and providing information about his start date. The email also includes instructions for reviewing and signing the contract, as well as requests for a staff introduction, headshot, and video of waving.',
 'Translation': 'El correo electrónico felicita a Edwin por unirse al equipo como Analista de Negocios y proporciona información sobre su fecha de inicio. El correo electrónico también incluye instrucciones para revisar y firmar el contrato, así como solicitudes de una introducción del personal, una foto de cabeza y un video saludando.'}

In [16]:
output_dict.get('Importance')

'Medium'

In [24]:
html_table=get_completion(
    f"""
    Translate the following python dictionary from JSON to an HTML \
    table with column headers and title: {output_dict}
    """
)
html_table

'<table>\n  <caption>Email Information</caption>\n  <tr>\n    <th>Sender</th>\n    <th>Tone</th>\n    <th>Deliverables</th>\n    <th>Number of deliverables</th>\n    <th>Tags</th>\n    <th>Importance</th>\n    <th>Summary</th>\n    <th>Translation</th>\n  </tr>\n  <tr>\n    <td>Stephanie Reynolds</td>\n    <td>Positive</td>\n    <td>Staff Introduction Example (PDF), Headshot/Candid Photo (.jpg), Video of waving (.mp4)</td>\n    <td>3</td>\n    <td>New Hire, Onboarding, Contract, Deliverables, Start Date</td>\n    <td>Medium</td>\n    <td>The email is congratulating Edwin on joining the team as a Business Analyst and providing information about his start date. The email also includes instructions for reviewing and signing the contract, as well as requests for a staff introduction, headshot, and video of waving.</td>\n    <td>El correo electrónico felicita a Edwin por unirse al equipo como Analista de Negocios y proporciona información sobre su fecha de inicio. El correo electrónico tamb

In [27]:
display(HTML(html_table))

Sender,Tone,Deliverables,Number of deliverables,Tags,Importance,Summary,Translation
Stephanie Reynolds,Positive,"Staff Introduction Example (PDF), Headshot/Candid Photo (.jpg), Video of waving (.mp4)",3,"New Hire, Onboarding, Contract, Deliverables, Start Date",Medium,"The email is congratulating Edwin on joining the team as a Business Analyst and providing information about his start date. The email also includes instructions for reviewing and signing the contract, as well as requests for a staff introduction, headshot, and video of waving.","El correo electrónico felicita a Edwin por unirse al equipo como Analista de Negocios y proporciona información sobre su fecha de inicio. El correo electrónico también incluye instrucciones para revisar y firmar el contrato, así como solicitudes de una introducción del personal, una foto de cabeza y un video saludando."
